In [2]:
import pandas as pd
from database import select_dataset
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import plotly.express as px

# Data import
wb_data = select_dataset('worldbank')
aqua_df = select_dataset('aquastat').fillna(0)
wb_data.Year = wb_data.Year.astype(int)
aqua_df.Year = aqua_df.Year.astype(int)

# Data selection
mwu = aqua_df.query("Variable == 'MWU' and Year == 2005")[['ISO', 'Value']]
wp  = aqua_df.query("Variable == 'WP' and Year == 2011")[['ISO', 'Value']]
pop = wb_data.query("Variable == 'Pop' and Year == 2005")[['ISO', 'Value']]
gdp = wb_data.query("Variable == 'GDP' and Year == 2005")[['ISO', 'Value']]

# Data processing
dfs = [df.set_index('ISO') for df in [mwu, wp, gdp, pop]]
df = pd.concat(dfs, axis=1).dropna()
df.columns = ['MWU', 'WP', 'GDP', 'POP']

df['GDPc'] = df['GDP'] / df['POP']
df['log_MWU'] = np.log(df['MWU'])
df['log_WP'] = np.log(df['WP'])
df['log_GDP'] = np.log(df['GDP'])
df['log_POP'] = np.log(df['POP'])
df['log_GDPc'] =  np.log(df['GDPc'])

# Regression
model = smf.ols(formula='log_MWU ~  log_WP + log_GDPc + log_POP', data=df)
model_fit = model.fit()
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                log_MWU   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.915
Method:                 Least Squares   F-statistic:                     201.8
Date:                Tue, 09 Feb 2021   Prob (F-statistic):           5.72e-29
Time:                        14:48:31   Log-Likelihood:                -43.622
No. Observations:                  57   AIC:                             95.24
Df Residuals:                      53   BIC:                             103.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.9831      0.872    -25.213      0.000     -23.732     -20.234
log_WP        -0.2099      0.094     -2.230      0.030      -0.399      -0.021
log_GDPc       0.6379      0.056     11.378      0.000       0.525       0.750
log_POP        1.6301      0.075     21.786      0.000       1.480       1.780
==============================================================================
Omnibus:                        0.549   Durbin-Watson:                   2.427
Prob(Omnibus):                  0.760   Jarque-Bera (JB):                0.214
Skew:                           0.142   Prob(JB):                        0.899
Kurtosis:                       3.097   Cond. No.                         220.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [2]:
# Plots
df['log_MWU_pred'] = model_fit.predict()
df['residuals'] = model_fit.resid

df['MWU_pred'] = np.exp(df['log_MWU_pred'])
df['MWU-MWU_pred'] = df['MWU_pred'] - df['MWU']

In [3]:
#px.scatter(df.reset_index(), x='log_MWU', y='log_MWU_pred', hover_data=['ISO'])

In [4]:
#px.scatter(df.reset_index(), x='log_MWU', y='residuals', hover_data=['ISO'])

In [5]:
#px.scatter(df.reset_index(), x='MWU', y='MWU_pred', hover_data=['ISO'])

In [7]:
#px.scatter(df.reset_index(), x='MWU', y='MWU-MWU_pred', hover_data=['ISO'])